In [1]:
import spacy
import json

In [4]:
with open("unstructured.txt", 'r') as file:
    lines = file.readlines()

source_list = [
    "Unit42",
    "Fireeye"
]

source_list = [source.lower() for source in source_list]

In [7]:
def check_new_entry(list_of_sources, line):
    first_substr = line.split(':')[0]

    is_new_entry = first_substr.lower() in list_of_sources

    return is_new_entry

def collect_ioc(entry_text):
    substr_format = r'\([^\(:\)]+: [^\(:\)]*(?:\([^\(:\)]*\)[^\(:\)]*)*\)'
    matches = re.findall(substr_format, entry_text)
    return matches

def collect_ioc(list_of_sources, lines):
    entries = []
    entry_text = lines[0]
    for line in lines[1:]:
        if check_new_entry(list_of_sources, line):
            entries.append(entry_text)
            entry_text = line
            source = line.split(':')[0]
        else:
            entry_text = entry_text + line
    entries.append(entry_text)

    return entries

def entry_to_dict(entry):
    substr_format = r'([^:]+):([^\s]+) - ([.\S\s]+)'
    match = re.search(substr_format, entry)
    if match:
        source = match.group(1).strip()
        name = match.group(2).strip()
        text = match.group(3).strip()
        list_ioc_dicts = []
        iocs = collect_ioc(text)
        
        for ioc in iocs:
            ioc_no_semicolon = ioc.replace(';', ':')
            ioc_elements = ioc_no_semicolon[1:-1].split(':')
            ioc_type = ioc_elements[0]
            ioc_value = ioc_elements[1][1:]
            list_ioc_dicts.append(
                {
                    'ioc_type':ioc_type,
                    'ioc_value':ioc_value
                }
            )

    entry_dict = {
        'name':name,
        'source':source,
        'ioc':list_ioc_dicts,
        'text':text
    }

    return entry_dict

data_dicts = []
for data in parsed_data:
    data_dicts.append(entry_to_dict(data))

for data_dict in data_dicts:
    source = data_dict['source']
    name = data_dict['name']
    output_fnmae = source + ', ' + name
    with open(output_fname, 'w') as outfile:
        json.dump(data_dict, outfile)


In [ ]:
f = open('unit42, ironnetinjector')
data = json.load(f)
data